# Report NoteBook

To do:  
   - Write a locustfile to stress test the http server
   - Verification event body

Extras


Done:
  - Send the records as json
  - Add all verifications to the different actions  
  - Garantee the Records from different actions will go to different places.  
  - Implement a reponse 200 
  - Verification if events in track are a list

# Main file

The next cell generates the main.py python script file.  

In [25]:
%%file main.py
from flask import Flask, request, jsonify,abort,make_response
import boto3
import json
'''
This python script should be executed in super user conditions to this execute the command: sudo python3 "folderpath"/main.py  
This scripts will create a http server with an Welcome message and 3 actions /track,/alias,/profile.

The intend of all the actions in this script are only to garantee that data gets stored

############################## /track ##############################

This action expects the data once the method that is defined is POST. The data expect by /track should be
something like this example:
{"userId": "fakeid01","events": [{"eventName": "test","metadata": {"test":1},"timestampUTC": 0}]}

Amazon Firehose Delivery Stream
If everything goes well it will put this record into the firehose delivery stream projectx-track which will write in file track
in amazon S3 after 300 seconds from the first put_record or after the amount of records reach to 5 MB.


############################## /alias ##############################

This action expects the data once the method that is defined is POST. The data expect by /alias should be
something like this example:
alias_user=  {  "newUserId": 'user02',"originalUserId": 'user01',"timestampUTC": 0}

Amazon Firehose Delivery Stream
If everything goes well it will put this record into the firehose delivery stream projectx-alias which will write in file alias
in amazon S3 after 300 seconds from the first put_record or after the amount of records reach to 5 MB.


############################# /profile #############################

This action expects the data once the method that is defined is POST. The data expect by /profile should be
something like this example:
user_profile= {   "userId": "string",  "attributes": {},  "timestampUTC": 0 }

Amazon Firehose Delivery Stream
If everything goes well it will put this record into the firehose delivery stream projectx-profile which will write in file profile
in amazon S3 after 300 seconds from the first put_record or after the amount of records reach to 5 MB.


'''

# Create a Flask applicatiob
app = Flask(__name__)

#Creates a client to interact with firhose
client_firehose = boto3.client('firehose',region_name='us-east-2')


#Welcome Message
@app.route('/')
def begin():
    return ("Welcome, This API as 3 actions track,alias,profile")



####### /track  #####

@app.route('/track',methods = ['POST'])
def track():
    
    ## Verify Json Format and fields
    if request.is_json:
        user_events = request.get_json()

        if type(user_events) ==dict:
            
            #Correct Json Fields Verification
            if list(user_events.keys())==['userId', 'events'] and type(user_events['events']) ==list:
                
                #creates a string with the json format of the request
                user_events_json=json.dumps(user_events)

                #Sends the string to firehose Delivery Stream projectx-track which will write periodically on amazon S3 on the preffix track
                response = client_firehose.put_record(DeliveryStreamName='projectx-track',Record={'Data': user_events_json})  
            
                res_fields = {"Description":'OK'}
                resp=make_response(json.dumps(res_fields),response['ResponseMetadata']['HTTPStatusCode'])
                return resp
            
            else:
                abort(400,'Wrong Json Fiels')
        else:
            abort(400,'request data was understood as '+str(type(user_events))+ ' Instead of Json')
    else:
        abort(400,'API was not able to recognize JSON format')


###### /alias #####

@app.route('/alias',methods = ['POST'])
def alias():
    

    ## Verify Json Format and fields
    if request.is_json:
        alias_user = request.get_json()

        if type(alias_user) ==dict:
            
    
            #Correct Json Fields Verification
            if list(alias_user.keys())==  ["newUserId","originalUserId","timestampUTC"]:

                #creates a string with the json format of the request
                alias_user_json=json.dumps(alias_user)

                #Sends the string to firehose Delivery Stream projectx-alias which will write periodically on amazon S3 on the preffix alias
                response = client_firehose.put_record(DeliveryStreamName='projectx-alias',Record={'Data': alias_user_json})
                res_fields = {"Description":'OK'}
                resp=make_response(json.dumps(res_fields),response['ResponseMetadata']['HTTPStatusCode'])
                return resp
            
            else:
                abort(400,'Error: Wrong Json Fiels')
        else:
            abort(400,'request data was understood as '+str(type(alias_user)) + ' Instead of Json')
    else:
        abort(400,'API was not able to recognize JSON format')


###### /profile #####

@app.route('/profile',methods = ['POST'])
def profile():    

    
    ## Verify Json Format and fields
    if request.is_json:
        user_profile = request.get_json()
        
        if type(user_profile) ==dict:
            
            #Correct Json Fields Verification
            if list(user_profile.keys())==  ["userId","attributes","timestampUTC"]:

                #creates a string with the json format of the request
                user_profile_json=json.dumps(user_profile)

                #Sends the string to firehose Delivery Stream projectx-profile which will write periodically on amazon S3 on the preffix profile
                response= client_firehose.put_record(DeliveryStreamName='projectx-profile',Record={'Data': user_profile_json})
                
                res_fields = {"Description":'OK'}
                resp=make_response(json.dumps(res_fields),response['ResponseMetadata']['HTTPStatusCode'])
                return resp
            else:
                abort(400,'Error: Wrong Json Fiels')
        else:
            abort(400,'request data was understood as '+ str(type(user_profile)) + ' Instead of Json')
    else:
        abort(400,'API was not able to recognize JSON format')



if __name__ == "__main__":
    app.run(host='0.0.0.0',port=80)

Overwriting main.py


#### Improvements to be done to the Main File:
   - Implement a better verification process.

#### Considerations to make regarding the Main File:  
   - One thing that should be considered is that, even though that a structure for the data was given, all the data should be store once supposedly all of our data come from the bound-touch application, and even though we might not be prepared to treat all of it and put in the right paths in bucket we should stored in order to not lose it.
 


# Test script

The next cell writes the test.py script

In [24]:
%%file test.py
import requests
import time
import json
httpaddress = 'http://18.219.203.248/'
'''
This is the test script
This Script is divided into 3 parts each one testing all the 3 actions for a specific scenario:
  1.Scenario - Testing the actions with the desired records - the requests sent are all with the desired format so everything sould run smooth 
  2.Scenario - Testing the actions responses with jsons but with wrong field names - 400 bad request response is expected with the message "Error: Wrong Json Fiels"
  3.Scenario - Testing the actions responses with data that is not in json format - 400 bad request response is expected with the error message "request data was understood as *type* Instead of Json"
'''



############ 1.Scenario ################


print('1.Scenario - Testing the actions with the desired records - 200 response is expected')

## Test /track
httpaddresstrack  = httpaddress + 'track'
user_events = {"userId": "fakeid01","events": [{"eventName": "test","metadata": {"test":1},"timestampUTC": 0}]}
response = requests.post(httpaddresstrack, json=user_events)
if response.status_code == 200:
    print(response)
    print(response.text)
else:
    print(response)
    print('/track is not processing the pretend json format')
    
## Test /alias
httpaddressalias  = httpaddress + 'alias'
originalUserId = 'user01'
new_used_Id  = 'nuser01'
alias_user=  {  "newUserId": new_used_Id,"originalUserId": originalUserId,"timestampUTC": time.time()}
response = requests.post(httpaddressalias, json=alias_user)
if response.status_code == 200:
    print(response)
    print(response.text)
else:
    print(response)
    print('/alias is not processing the pretend json format')

## Test /profile
httpaddressprofile  = httpaddress + 'profile'
user_profile= {   "userId": "string",  "attributes": {},  "timestampUTC": 0 }
response = requests.post(httpaddressprofile, json=user_profile)
if response.status_code == 200:
    print(response)
    print(response.text)
else:
    print(response)
    print('/track is not processing the pretend json format')

    

    
############ 2.Scenario ################


print('2.Scenario - Testing the actions responses with jsons but with wrong field names - 400 bad request response is expected ')

## Test /track
httpaddresstrack  = httpaddress + 'track'
user_events = {"userId123": "fakeid01","events": [{"eventName": "test","metadata": {"test":1},"timestampUTC": 0}]}
response = requests.post(httpaddresstrack, json=user_events)
print(response)
print(response.content)
    
## Test /alias
httpaddressalias  = httpaddress + 'alias'
originalUserId = 'user01'
new_used_Id  = 'nuser01'
alias_user=  {  "newUserId131": new_used_Id,"originalUserId": originalUserId,"timestampUTC": time.time()}
response = requests.post(httpaddressalias, json=alias_user)
print(response)
print(response.content)

## Test /profile
httpaddressprofile  = httpaddress + 'profile'
user_profile= {   "userId131": "string",  "attributes": {},  "timestampUTC": 0 }
response = requests.post(httpaddressprofile, json=user_profile)
print(response)
print(response.content)




############ 3.Scenario ################


print('3.Scenario - Testing the actions responses with data that is not in json format - 400 bad request response is expected')

httpaddresstrack  = httpaddress + 'track'
user_events = "aifbaw egiqedfjvcpal"
response = requests.post(httpaddresstrack, json=user_events)
print(response)
print(response.content)
    
## Test /alias
httpaddressalias  = httpaddress + 'alias'
originalUserId = 'user01'
new_used_Id  = 'nuser01'
alias_user=  "aifbaw egiqedfjvcpal"
response = requests.post(httpaddressalias, json=alias_user)
print(response)
print(response.content)

## Test /profile
httpaddressprofile  = httpaddress + 'profile'
user_profile="aifbaw egiqedfjvcpal"
response = requests.post(httpaddressprofile, json=user_profile)
print(response)
print(response.content)


Overwriting test.py


#### Improvements to be done to the Test File:
   - Clearly this is not how I would imagine how a good test file would be. Probably a lot of it needs to be redesigned, maybe by creating functions and actions as response of an interaction with the user in the comand line. But well, this function does part of job and can test most of the implemented responses.


## Exploring some boto3 features

In [21]:
import boto3
client = boto3.client('firehose', region_name =  'us-east-2')

In [22]:
client.list_delivery_streams()

{'DeliveryStreamNames': ['projectx-alias',
  'projectx-profile',
  'projectx-track',
  'projectx_del'],
 'HasMoreDeliveryStreams': False,
 'ResponseMetadata': {'RequestId': 'f68d8ac5-c356-022d-a0e9-6765a9603396',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f68d8ac5-c356-022d-a0e9-6765a9603396',
   'x-amz-id-2': 'iD0nW0VMBZTUX859JprAd+jDti0v8D/vxWwxQ5DOVoy9z10B3zWUUzJTty0bQVqfL4Lhjaok573DG4XG6o2hOjRquKmO3JPz',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '124',
   'date': 'Fri, 06 Dec 2019 23:59:28 GMT'},
  'RetryAttempts': 0}}

In [23]:
client.describe_delivery_stream(DeliveryStreamName='projectx_del')

{'DeliveryStreamDescription': {'DeliveryStreamName': 'projectx_del',
  'DeliveryStreamARN': 'arn:aws:firehose:us-east-2:841513907133:deliverystream/projectx_del',
  'DeliveryStreamStatus': 'ACTIVE',
  'DeliveryStreamEncryptionConfiguration': {'Status': 'DISABLED'},
  'DeliveryStreamType': 'DirectPut',
  'VersionId': '2',
  'CreateTimestamp': datetime.datetime(2019, 12, 5, 16, 42, 28, 329000, tzinfo=tzlocal()),
  'LastUpdateTimestamp': datetime.datetime(2019, 12, 5, 19, 3, 18, 217000, tzinfo=tzlocal()),
  'Destinations': [{'DestinationId': 'destinationId-000000000001',
    'S3DestinationDescription': {'RoleARN': 'arn:aws:iam::841513907133:role/firehose_delivery_role',
     'BucketARN': 'arn:aws:s3:::projectxs3',
     'Prefix': '',
     'ErrorOutputPrefix': '',
     'BufferingHints': {'SizeInMBs': 5, 'IntervalInSeconds': 300},
     'CompressionFormat': 'UNCOMPRESSED',
     'EncryptionConfiguration': {'NoEncryptionConfig': 'NoEncryption'},
     'CloudWatchLoggingOptions': {'Enabled': True

In [14]:
httpaddress = 'http://18.219.203.248/'
print('Testing the used with good records')
httpaddresstrack  = httpaddress + 'track'
user_events = {"userId": "fakeid01","events": [{"eventName": "test","metadata": {"test":1},"timestampUTC": 0}]}
response = requests.post(httpaddresstrack, json=user_events)

Testing the used with good records


In [16]:
response.status_code

200